# MOST OF THIS CODE WAS COPIED FROM LECTURE 2.2. THE PIECES OF CODE WHICH HAVE NOT BEEN COPIED WILL BE POINTED OUT AS PART OF THE COMMENTS

In [1]:
#This cell imports all the correct libraries to run the code successfully

from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
lem = WordNetLemmatizer()

In [2]:
#This cell loads the corpus which comes as a CSV file, into a pandas dataframe. It was copied from lecture 2.2

blog_df = pd.read_csv('data/blogtext.csv', encoding='utf-8')
blog_df.shape #<--This part shows how many words are contained in the entire document and how many columns the 
              #CSV file contains

(681284, 7)

In [3]:
#This cell provides a sample of the first 20 entries in the pandas dataframe

blog_df.head(20)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


In [4]:
#This is the tokeniser, as explained in the critical report. This code was copied from lecture 2.2.

def my_tokeniser(doc):
    #Split on spaces
    tokens = re.split(r'[-\s.,;!?]+', doc)
    processed = []
    for t in tokens:
        #Lemmatise and make lowercase
        t = lem.lemmatize(t.lower())
        #Remove stop words
        if not t in _stop_words.ENGLISH_STOP_WORDS:
            processed = processed + [t]
    #Return an array of tokens for that document
    return processed

# Zodiac 

In [5]:
#I wrote the code in this cell myself. As explained in the critical report this takes a slice of the corpus
#by star sign to analyse. It's a bit clunky but it works.

aquarius_text = [] #<-- The entries for the aquarius star sign are loaded in this list. The same is repeated
aries_text = []    #for every star sign.
cancer_text = []
capricorn_text = []
gemini_text = []
leo_text = []
libra_text = []
pisces_text = []
sagittarus_text = []
scorpio_text = []
taurus_text = []
virgo_text = []

for i, item in enumerate(blog_df.sign):
    if i >= 40000 and i <= 50000:       #<-- This is the part of the code doing the actual slice
        if item == 'Aquarius':
            aquarius_text.append(blog_df[blog_df.sign == 'Aquarius'].text[i]) #<-- This appends all the entries
        elif item == 'Aries':                                                 #for aquarius star sign to the
            aries_text.append(blog_df[blog_df.sign == 'Aries'].text[i])       #aquarius list above. The same is
        elif item == 'Cancer':                                                #repeated for every star sign.
            cancer_text.append(blog_df[blog_df.sign == 'Cancer'].text[i])
        elif item == 'Capricorn':
            capricorn_text.append(blog_df[blog_df.sign == 'Capricorn'].text[i])
        elif item == 'Gemini':
            gemini_text.append(blog_df[blog_df.sign == 'Gemini'].text[i])
        elif item == 'Leo':
            leo_text.append(blog_df[blog_df.sign == 'Leo'].text[i])
        elif item == 'Libra':
            libra_text.append(blog_df[blog_df.sign == 'Libra'].text[i])
        elif item == 'Pisces':
            pisces_text.append(blog_df[blog_df.sign == 'Pisces'].text[i])
        elif item == 'Sagittarus':
            sagittarus_text.append(blog_df[blog_df.sign == 'Sagittarus'].text[i])
        elif item == 'Scorpio':
            scorpio_text.append(blog_df[blog_df.sign == 'Scorpio'].text[i])
        elif item == 'Taurus':
            taurus_text.append(blog_df[blog_df.sign == 'Taurus'].text[i])
        elif item == 'Virgo':
            virgo_text.append(blog_df[blog_df.sign == 'Virgo'].text[i])

In [6]:
# I wrote the code in this cell myself. It casts all of the list from the cell above into strings and stores them
#into variables of the same name. On second thoughts I should have called them something different. The code on this
# cell is necessary but I cannot remember why.

aquarius_text = str(aquarius_text)
aries_text = str(aries_text)
cancer_text = str(cancer_text)
capricorn_text = str(capricorn_text)
gemini_text = str(gemini_text)
leo_text = str(leo_text)
libra_text = str(libra_text)
pisces_text = str(pisces_text)
sagittarus_text = str(sagittarus_text)
scorpio_text = str(scorpio_text)
taurus_text = str(taurus_text)
virgo_text = str(virgo_text)

In [7]:
#I wrote the code in this cell myself. This code concatenates all of the strings from the cell above into one long
#string separated by three newlines so as to make it easier to split into chapters.

sign_text = aquarius_text + "\n\n\n" + aries_text + "\n\n\n" + cancer_text\
            + "\n\n\n" + capricorn_text + "\n\n\n" + gemini_text\
            + "\n\n\n" + leo_text + "\n\n\n" + libra_text + "\n\n\n" + pisces_text\
            + "\n\n\n" + sagittarus_text + "\n\n\n" + scorpio_text\
            + "\n\n\n" + taurus_text + "\n\n\n" + virgo_text

In [8]:
#I wrote this code myself. This a regex that splits the long string from above into chapters and stores them into
#a variable called signs_occupation. 

signs_occupation = re.split(r'\n\n\n', sign_text)

# ZODIAC BOW TF

In [9]:
#Using the CountVectorizer to get a bag of words using a custom tokeniser. This code was copied from lecture 2.2,
#it tokenises all the words from each chapter into vectors BoW as explained in the critical report.

count_vectoriser = CountVectorizer(tokenizer=my_tokeniser)
bag_of_words = count_vectoriser.fit_transform(signs_occupation)
print(bag_of_words.todense().shape) #<--This shows how many documents there are, and the number of words contained
                                    #across all documents.

(12, 71632)


In [10]:
#This code was copied from lecture 2.2. I don't exaclty what it does but I believe it counts the TF from the BoW, as
#explained in the critical report.

vocab = count_vectoriser.get_feature_names_out()
bag_of_words_df = pd.DataFrame(bag_of_words.todense(), columns = vocab)

In [11]:
#Topic 20 most common words (we've already removed STOP WORDS). This code was copied from lecture 2.2 although I
#added some entries to it. It displays the 20 least common words from the blog entries according to star signs

print(bag_of_words_df.iloc[0].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[1].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[2].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[3].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[4].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[5].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[6].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[7].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[8].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[9].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[10].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[11].sort_values()[-20:])

state           474
learning        479
time            485
president       504
literacy        520
la              590
&               592
&nbsp           669
education       695
ha              697
library         804
digest          843
school          875
eric            975
information    1003
student        1042
'              1214
urllink        1350
wa             1374
"              1607
Name: 0, dtype: int64 
 ha        159
it's      174
want      182
day       185
love      189
think     202
got       206
thing     206
really    212
time      238
don't     263
know      276
people    281
'         308
&nbsp     341
like      392
i'm       443
just      465
"         580
wa        671
Name: 1, dtype: int64 
 ha          547
people      550
urllink     567
really      620
got         634
thing       641
don't       650
day         671
know        680
good        697
think       745
'           906
it's        908
time       1127
i'm        1181
like       1206
&nbsp      1246


Topic 0 = aquarius<br>
Topic 1 = aries<br>
Topic 2 = cancer<br>
Topic 3 = capricorn<br>
Topic 4 = gemini<br>
Topic 5 = leo<br>
Topic 6 = libra<br>
Topic 7 = pisces<br>
Topic 8 = sagittarus<br>
Topic 9 = scorpio<br>
Topic 10 = taurus<br>
Topic 11 = virgo<br>

# ZODIAC TFIDF

In [12]:
#Using the TFIDF Vectorizer to get TFIDF vectors with custom tokeniser. This code was copied from lecture 2.2
#it tokenises all the words from each chapter into TF-IDF vectors as explained in the critical report

tfidf_vectoriser = TfidfVectorizer(tokenizer=my_tokeniser)
tfidf = tfidf_vectoriser.fit_transform(signs_occupation)
print(tfidf.todense().shape)

(12, 71632)


In [13]:
#This code was copied from lecture 2.2. It displays the TF-IDF weight values for the least common vectors for
#each document. according to star sign

tfidf_df = pd.DataFrame(tfidf.todense(), columns = vocab)
for i in range(len(tfidf_df)):
    print("Signs occupation", i)
    print(tfidf_df.iloc[i].sort_values(ascending = False).head(20))

Signs occupation 0
"              0.278510
digest         0.264529
wa             0.238129
urllink        0.233970
eric           0.232415
'              0.210399
literacy       0.181793
student        0.180590
information    0.173831
school         0.151647
digest:        0.141013
library        0.139342
ha             0.120798
education      0.120451
&nbsp          0.115945
librarian      0.113620
los            0.108597
&              0.102600
la             0.102253
president      0.087349
Name: 0, dtype: float64
Signs occupation 1
wa        0.367346
"         0.317527
just      0.254569
i'm       0.242525
like      0.214605
&nbsp     0.186684
spense    0.173226
'         0.168618
people    0.153836
know      0.151099
don't     0.143982
time      0.130296
really    0.116062
got       0.112777
thing     0.112777
think     0.110587
love      0.103470
day       0.101280
want      0.099638
it's      0.095258
Name: 1, dtype: float64
Signs occupation 2
wa       0.406212
"        0.255078

Topic 0 = aquarius<br>
Topic 1 = aries<br>
Topic 2 = cancer<br>
Topic 3 = capricorn<br>
Topic 4 = gemini<br>
Topic 5 = leo<br>
Topic 6 = libra<br>
Topic 7 = pisces<br>
Topic 8 = sagittarus<br>
Topic 9 = scorpio<br>
Topic 10 = taurus<br>
Topic 11 = virgo<br>

# OCCUPATIONS

In [14]:
#I wrote the code in this cell myself. As explained in the critical report this takes a slice of the corpus
#by occupation to analyse. It's a bit clunky but it works.

student_text = []
technology_text = []
arts_text = []
education_text = []
communications_media_text = []
internet_text = []
non_profit_text = []
engineering_text = []
law_text = []
publishing_text = []
science_text = []
government_text = []

for i, item in enumerate(blog_df.topic):
    if i >= 40000 and i <= 50000:        #<-- This is the part of the code doing the actual slice
        if item == 'Student':
            student_text.append(blog_df[blog_df.topic == item].text[i])   #<-- This appends all the entries
        elif item == 'Technology':                                        #for student occupation to the list above
            technology_text.append(blog_df[blog_df.topic == item].text[i]) #the same is repeated for every occupation
        elif item == 'Arts':
            arts_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Education':
            education_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Communications-Media':
            communications_media_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Internet':
            internet_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Non-Profit':
            non_profit_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Engineering':
            engineering_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Law':
            law_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Publishing':
            publishing_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Science':
            science_text.append(blog_df[blog_df.topic == item].text[i])
        elif item == 'Government':
            government_text.append(blog_df[blog_df.topic == item].text[i])

In [15]:
# I wrote the code in this cell myself. It casts all of the list from the cell above into strings and stores them
#into variables of the same name. On second thoughts I should have called them something different. The code on this
# cell is necessary but I cannot remember why.

student_text = str(student_text)
technology_text = str(technology_text)
arts_text = str(arts_text)
education_text = str(education_text)
communications_media_text = str(communications_media_text)
internet_text = str(internet_text)
non_profit_text = str(non_profit_text)
engineering_text = str(engineering_text)
law_text = str(law_text)
publishing_text = str(publishing_text)
science_text = str(science_text)
government_text = str(government_text)

In [16]:
#I wrote the code in this cell myself. This code concatenates all of the strings from the cell above into one long
#string separated by three newlines so as to make it easier to split into chapters.

topic_text = student_text + "\n\n\n" + technology_text + "\n\n\n" + arts_text + "\n\n\n" + education_text\
            + "\n\n\n" + communications_media_text + "\n\n\n" + internet_text + "\n\n\n" + non_profit_text\
            + "\n\n\n" + engineering_text + "\n\n\n" + law_text + "\n\n\n" + publishing_text\
            + "\n\n\n" + science_text + "\n\n\n" + government_text

In [17]:
#I wrote this code myself. This a regex that splits the long string from above into chapters and stores them into
#a variable called signs_occupation.

topics_occupation = re.split(r'\n\n\n', topic_text)

# OCCUPATIONS BOW TF

In [18]:
#Using the CountVectorizer to get a bag of words using a custom tokeniser. This code was copied from lecture 2.2,
#it tokenises all the words from each chapter into vectors BoW as explained in the critical report.

count_vectoriser = CountVectorizer(tokenizer=my_tokeniser)
bag_of_words = count_vectoriser.fit_transform(topics_occupation)
print(bag_of_words.todense().shape) #<--This shows how many documents there are, and the number of words contained
                                    #across all documents.

(12, 52484)


In [19]:
#This code was copied from lecture 2.2. I don't exaclty what it does but I believe it counts the TF from the BoW, as
#explained in the critical report.

vocab = count_vectoriser.get_feature_names_out()
bag_of_words_df = pd.DataFrame(bag_of_words.todense(), columns = vocab)

In [20]:
#Topic 20 most common words (we've already removed STOP WORDS). This code was copied from lecture 2.2 although I
#added some entries to it. It displays the 20 least common words from the blog entries according to occupation

print(  bag_of_words_df.iloc[0].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[1].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[2].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[3].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[4].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[5].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[6].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[7].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[8].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[9].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[10].sort_values()[-20:],'\n',\
        bag_of_words_df.iloc[11].sort_values()[-20:])
        

going     1038
thing     1052
&nbsp     1097
it's      1120
people    1229
think     1238
good      1269
went      1275
time      1330
day       1380
know      1393
don't     1424
really    1470
got       1513
like      2045
just      2148
i'm       2357
'         2656
wa        4125
"         4442
Name: 0, dtype: int64 
 really      53
new         54
want        56
think       57
good        61
don't       64
day         73
know        74
like        87
ha          89
hakx        95
it's        95
time       105
just       106
brought    106
i'm        108
wa         242
'          284
urllink    305
"          441
Name: 1, dtype: int64 
 make        75
really      76
i've        76
way         77
it's        86
know        88
ha          93
life        98
game        99
urllink    109
thing      110
like       113
people     119
just       120
&nbsp      135
i'm        137
'          147
time       163
"          223
wa         270
Name: 2, dtype: int64 
 new             420
research

Topic 0 = student<br>
Topic 1 = technology<br>
Topic 2 = arts<br>
Topic 3 = education<br>
Topic 4 = communications_media<br>
Topic 5 = internet<br>
Topic 6 = non_profit<br>
Topic 7 = engineering<br>
Topic 8 = law<br>
Topic 9 = publishing<br>
Topic 10 = science<br>
Topic 11 = government<br>

# OCCUPATIONS TFIDF

In [21]:
#Using the TFIDF Vectorizer to get TFIDF vectors with custom tokeniser. This code was copied from lecture 2.2
#it tokenises all the words from each chapter into TF-IDF vectors as explained in the critical report

tfidf_vectoriser = TfidfVectorizer(tokenizer=my_tokeniser)
tfidf = tfidf_vectoriser.fit_transform(topics_occupation)#(chapters)
print(tfidf.todense().shape)

(12, 52484)


In [22]:
#This code was copied from lecture 2.2. It displays the TF-IDF weight values for the least common vectors for
#each document according to occupation.

tfidf_df = pd.DataFrame(tfidf.todense(), columns = vocab)
for i in range(len(tfidf_df)):
    print("topics occupation", i)
    print(tfidf_df.iloc[i].sort_values(ascending = False).head(20))

topics occupation 0
"         0.390974
wa        0.363073
'         0.233775
i'm       0.207458
just      0.189062
like      0.179996
really    0.139742
got       0.133171
don't     0.125337
know      0.122609
day       0.121464
den       0.119829
time      0.117063
went      0.112222
good      0.111694
think     0.108966
people    0.108174
&nbsp     0.104284
it's      0.098580
today     0.095538
Name: 0, dtype: float64
topics occupation 1
"                0.458087
urllink          0.342176
'                0.295004
hakx             0.283392
wa               0.251377
ebot2004:        0.140204
gremlinchief:    0.140204
brought          0.118920
i'm              0.112185
just             0.110107
time             0.109068
it's             0.098681
ha               0.092448
like             0.090371
cigar            0.083734
node             0.076944
know             0.076867
day              0.075828
ip               0.071733
wireless         0.068150
Name: 1, dtype: float64
topics occup

Topic 0 = student<br>
Topic 1 = technology<br>
Topic 2 = arts<br>
Topic 3 = education<br>
Topic 4 = communications_media<br>
Topic 5 = internet<br>
Topic 6 = non_profit<br>
Topic 7 = engineering<br>
Topic 8 = law<br>
Topic 9 = publishing<br>
Topic 10 = science<br>
Topic 11 = government<br>